In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from tokenizers import ByteLevelBPETokenizer
import my_predictor as mp
import gdi_loader as gdi
from sklearn import svm
import bert_swiss_lm as bsl
import itertools
import torch
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [2]:
my_predictor = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None)

In [3]:
my_predictor.fit()

In [4]:
my_predictor.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.91      0.93      0.92      3750
          BS       0.90      0.88      0.89      3269
          LU       0.92      0.85      0.88      3390
          ZH       0.89      0.95      0.92      3870

    accuracy                           0.90     14279
   macro avg       0.91      0.90      0.90     14279
weighted avg       0.91      0.90      0.90     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.63      0.62      0.62      1053
          BS       0.71      0.59      0.65      1528
          LU       0.61      0.47      0.53      1017
          ZH       0.56      0.86      0.68       932

    accuracy                           0.63      4530
   macro avg       0.63      0.64      0.62      4530
weighted avg       0.64      0.63      0.62      4530



In [3]:
my_predictor_pooler = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                     ["sentence_embedding_first_state", "audio"],
                                     audio_length=10)

my_predictor_pooler.fit()
my_predictor_pooler.predict_all()

Training set:
              precision    recall  f1-score   support

          BE       0.78      0.83      0.81      3750
          BS       0.74      0.73      0.73      3269
          LU       0.74      0.67      0.71      3390
          ZH       0.80      0.82      0.81      3870

    accuracy                           0.77     14279
   macro avg       0.76      0.76      0.76     14279
weighted avg       0.77      0.77      0.77     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.57      0.60      0.58      1053
          BS       0.68      0.54      0.60      1528
          LU       0.53      0.45      0.49      1017
          ZH       0.57      0.81      0.67       932

    accuracy                           0.59      4530
   macro avg       0.59      0.60      0.58      4530
weighted avg       0.60      0.59      0.59      4530



In [8]:
my_predictor_with_audio = mp.MyPredictor("train_embedding_bert_swiss_lm.feather", "dev_embedding_bert_swiss_lm.feather", None, "sentence_embedding_pooler")
my_predictor_with_audio.fit()
my_predictor_with_audio.predict_all()

KeyError: 's'

In [3]:
my_predictor_pooler.fit()

In [3]:
for i in (0.00006,):
    # best C: 0.00006 (macro f1 0.33)
    # standard_Scaler= True: 0.0006 (accuracy 0.31) schlechter als 0.00006 (accuracy 0.38)
    # standard_Scaler= False: 0.0006 (accuracy )  als normalize_each_vector=False 0.00006 (accuracy 0.31)
#    for standard_scalar in (False, True):
    for kernel in ("linear",):
        standard_scalar = True
        norm = False
        my_predictor_audio = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                            ("audio",),
                                    normalize_each_vector=norm)

        my_predictor_audio.fit(C=i, standard_scaler=standard_scalar, kernel=kernel)
        print("C: ", i, "kernel: ", kernel)
        my_predictor_audio.predict_all()

C:  6e-05 kernel:  linear
Training set:
              precision    recall  f1-score   support

          BE       0.99      0.99      0.99      3750
          BS       0.99      0.98      0.99      3269
          LU       0.99      1.00      0.99      3390
          ZH       0.99      0.99      0.99      3870

    accuracy                           0.99     14279
   macro avg       0.99      0.99      0.99     14279
weighted avg       0.99      0.99      0.99     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.20      0.24      0.22      1053
          BS       0.62      0.39      0.48      1528
          LU       0.29      0.14      0.18      1017
          ZH       0.38      0.73      0.50       932

    accuracy                           0.37      4530
   macro avg       0.37      0.37      0.34      4530
weighted avg       0.40      0.37      0.36      4530



In [8]:
for standard_scalar in (False,):
    for norm in (False,):
        my_predictor_audio_gaussian = mp.MyPredictor("train_embedding_bert_swiss_lm.feather",
                                     "dev_embedding_bert_swiss_lm.feather",
                                     None,
                                                     ("sentence_embedding_first_state", "audio"),
                                    normalize_each_vector=norm)

        my_predictor_audio_gaussian.fit_gaussian_naive_bayes(standard_scaler=standard_scalar)
        print("standard_scaler: ", standard_scalar, "normalize_each_vector: ", norm)
        my_predictor_audio_gaussian.predict_all()

standard_scaler:  False normalize_each_vector:  False
Training set:
              precision    recall  f1-score   support

          BE       0.85      0.84      0.84      3750
          BS       0.80      0.82      0.81      3269
          LU       0.82      0.75      0.78      3390
          ZH       0.81      0.87      0.84      3870

    accuracy                           0.82     14279
   macro avg       0.82      0.82      0.82     14279
weighted avg       0.82      0.82      0.82     14279

Dev set:
              precision    recall  f1-score   support

          BE       0.66      0.59      0.62      1053
          BS       0.80      0.77      0.79      1528
          LU       0.81      0.57      0.67      1017
          ZH       0.60      0.90      0.72       932

    accuracy                           0.71      4530
   macro avg       0.72      0.71      0.70      4530
weighted avg       0.73      0.71      0.71      4530



0.6147505478251539
              precision    recall  f1-score   support

          BE       0.60      0.62      0.61       242
          BS       0.69      0.52      0.60       335
          LU       0.69      0.49      0.57       235
          ZH       0.48      0.84      0.61       188

    accuracy                           0.60      1000
   macro avg       0.61      0.62      0.60      1000
weighted avg       0.63      0.60      0.60      1000



In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB

for i in (1000,):
    tokenizer = ByteLevelBPETokenizer()
    df = pd.read_feather("train_embedding_bert_swiss_lm.feather")
    tokenizer.train_from_iterator(df['text'].tolist(), vocab_size=i, min_frequency=2)
    encoded = tokenizer.encode_batch(df['text'].tolist())
    tokenized = [e.tokens for e in encoded]
    vectorizer = TfidfVectorizer()
    tfidf_weights = vectorizer.fit_transform([' '.join(tokens) for tokens in tokenized]).toarray()
    gaus = svm.LinearSVC()
    #gaus = svm.SVC(kernel='linear')
    #gaus = GaussianNB()
    #gaus = make_pipeline(StandardScaler(), gaus)
    gaus.fit(tfidf_weights, df['label'].tolist())
    df_dev = pd.read_feather("dev_embedding_bert_swiss_lm.feather")
    dev_tokenized = [e.tokens for e in tokenizer.encode_batch(df_dev['text'].tolist())]
    dev_vectorized = vectorizer.transform([' '.join(tokens) for tokens in dev_tokenized]).toarray()
    dev_predict = gaus.predict(dev_vectorized)
  #  train_predict = gaus.predict(tfidf_weights)
    #print(classification_report(df['label'].tolist(), train_predict))
    print(i)
    print(classification_report(df_dev['label'].tolist(), dev_predict))
    print(gaus.decision_function(dev_vectorized)[0:10])
    print(gaus.predict(dev_vectorized)[0:10])





1000
              precision    recall  f1-score   support

          BE       0.57      0.60      0.58      1053
          BS       0.73      0.60      0.66      1528
          LU       0.60      0.47      0.53      1017
          ZH       0.56      0.82      0.67       932

    accuracy                           0.62      4530
   macro avg       0.62      0.62      0.61      4530
weighted avg       0.63      0.62      0.61      4530

[[-1.31001618 -0.7810659   0.86600119 -2.04481747]
 [-0.44720302  0.20401442 -1.02759569 -0.91909179]
 [-0.02500046 -0.37267663  0.1809242  -3.47945755]
 [ 0.86143844 -1.7048028  -0.58103523 -2.71862425]
 [ 0.53552571 -1.86106865 -0.65354833 -1.24230647]
 [-1.77664344 -1.62984643 -2.90404613  2.20583745]
 [-1.63378788 -0.85808199 -0.08675167  0.08348658]
 [-0.80958421 -2.19490501  1.04960575 -2.02077076]
 [-1.81817499  1.28206112 -0.93882105 -1.71235151]
 [-0.34390398 -1.44972304 -0.24034991 -1.36355317]]
['LU' 'BS' 'LU' 'BE' 'BE' 'ZH' 'ZH' 'LU' 'BS' 

In [2]:
# For sentance_embedding and audio, GaussianNB is the best
# For ByteLevelBPETokenizer, LinearSVC is the best
# One can call LinearSVC.decision_function() to get features which can be used for classification

In [3]:
import swiss_dialect_predictors as sdp

In [4]:
train_df = pd.read_feather("train_embedding_bert_swiss_lm.feather")
dev_df = pd.read_feather("dev_embedding_bert_swiss_lm.feather")
train_swissbert = pd.read_feather("train_embedding_swissbert.feather")
dev_swissbert = pd.read_feather("dev_embedding_swissbert.feather")

In [5]:

from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
svm_gaussiannb_predictor = sdp.SwissDialectPredictorSvmGaussianNB( enable_audio=True,
                                                                   enable_sentance_embedding=True,
                                                                   enable_byte_pair_tfidf=True,
                                                                   normalize_each_vector=True,
                                                                   last_classifier=GaussianNB())
                                                                   #last_classifier=make_pipeline(StandardScaler(), TruncatedSVD(n_components=220), LinearSVC()))
svm_gaussiannb_predictor.fit(train_df)
train_predicted = svm_gaussiannb_predictor.predict(train_df)
dev_predicted = svm_gaussiannb_predictor.predict(dev_df)
# calculate f1 macro
print(classification_report(train_df['label'].tolist(), train_predicted))
print(classification_report(dev_df['label'].tolist(), dev_predicted))




tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.87      0.85      0.86      3750
          BS       0.82      0.84      0.83      3269
          LU       0.84      0.78      0.81      3390
          ZH       0.82      0.87      0.85      3870

    accuracy                           0.84     14279
   macro avg       0.84      0.84      0.84     14279
weighted avg       0.84      0.84      0.84     14279

              precision    recall  f1-score   support

          BE       0.67      0.62      0.64      1053
          BS       0.81      0.79      0.80      1528
          LU       0.82      0.58      0.68      1017
          ZH       0.62      0.89      0.73       932

    accuracy                           0.72      4530
   macro avg       0.73      0.72      0.71      4530
weighted avg       0.74      0.72      0.72      4530



              precision    recall  f1-score   support

          BE       0.43      0.73      0.54      1053
          BS       0.72      0.56      0.63      1528
          LU       0.70      0.27      0.39      1017
          ZH       0.51      0.63      0.56       932

    accuracy                           0.55      4530
   macro avg       0.59      0.55      0.53      4530
weighted avg       0.60      0.55      0.54      4530



In [6]:
from byte_pair_tfidf_vectorizer import BytePairTfidfVectorizer

byte_pair_tfidf_vectorizer = BytePairTfidfVectorizer(vocab_size=1000, min_frequency=2)
byte_pair_vectorized = byte_pair_tfidf_vectorizer.fit_transform(train_df["text"].tolist())

svm_linear = make_pipeline(StandardScaler(),TruncatedSVD(n_components=50), KNeighborsClassifier(n_neighbors=50))
svm_linear.fit(byte_pair_vectorized, train_df["label"].tolist())
prediction = svm_linear.predict(byte_pair_vectorized)
print(classification_report(train_df["label"].tolist(), prediction))
prediction_dev = svm_linear.predict(byte_pair_tfidf_vectorizer.transform(dev_df["text"].tolist()))
print(classification_report(dev_df["label"].tolist(), prediction_dev))




tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.55      0.82      0.66      3750
          BS       0.72      0.46      0.56      3269
          LU       0.71      0.49      0.58      3390
          ZH       0.64      0.69      0.66      3870

    accuracy                           0.63     14279
   macro avg       0.65      0.62      0.62     14279
weighted avg       0.65      0.63      0.62     14279

              precision    recall  f1-score   support

          BE       0.38      0.66      0.48      1053
          BS       0.76      0.41      0.53      1528
          LU       0.55      0.30      0.39      1017
          ZH       0.46      0.66      0.54       932

    accuracy                           0.49      4530
   macro avg       0.54      0.51      0.49      4530
weighted avg       0.57      0.49      0.49      4530



In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
for enable_byte_pair_tfidf in [True, False]:
    for enable_sentence_embedding in [True, False]:
        for enable_audio in [True, False]:
            if not(enable_audio or enable_sentence_embedding or enable_sentence_embedding):
                continue
            print("enable_byte_pair_tfidf: ", enable_byte_pair_tfidf, "enable_sentence_embedding: ", enable_sentence_embedding, "enable_audio: ", enable_audio)
            seperate_gaussian = sdp.SwissDialectPredictorSeperateGaussians(classifiers={
                                                                    "audio": make_pipeline(GaussianNB()),
                                                                    "sentene_embedding": make_pipeline(
#                                                                        StandardScaler(),
#                                                                        TruncatedSVD(400),
                                                                        GaussianNB()),
                                                                    "byte_pair_tfidf": svm.LinearSVC()},
                                                                normalize_each_vector=False,
                                                                enable_byte_pair_tfidf=enable_byte_pair_tfidf,
                                                                enable_sentance_embedding=enable_sentence_embedding,
                                                                enable_audio=enable_audio,
                                                               last_classifier=GaussianNB(), audio_weight=1)
            seperate_gaussian.fit(train_df)
            prediction_train = seperate_gaussian.predict(train_df)
            print(classification_report(train_df["label"].tolist(), prediction_train))
            prediction_dev = seperate_gaussian.predict(dev_df)
            print(classification_report(dev_df["label"].tolist(), prediction_dev))

enable_byte_pair_tfidf:  True enable_sentence_embedding:  True enable_audio:  True



tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.85      0.88      0.86      3750
          BS       0.98      0.73      0.83      3269
          LU       0.92      0.69      0.79      3390
          ZH       0.69      0.97      0.80      3870

    accuracy                           0.82     14279
   macro avg       0.86      0.82      0.82     14279
weighted avg       0.85      0.82      0.82     14279

              precision    recall  f1-score   support

          BE       0.59      0.58      0.59      1053
          BS       0.93      0.71      0.80      1528
          LU       0.83      0.39      0.53      1017
          ZH       0.49      0.97      0.65       932

    accuracy                           0.66      4530
   macro avg       0.71      0.66      0.64      4530
weighted avg       0.74      0.66      0.66      4530

enable_byte_pair_tfid

In [21]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import make_pipeline
for enable_byte_pair_tfidf in [True, False]:
    for enable_sentence_embedding in [True, False]:
        for enable_audio in [True, False]:
            if not(enable_audio or enable_sentence_embedding or enable_sentence_embedding):
                continue
            print("enable_byte_pair_tfidf: ", enable_byte_pair_tfidf, "enable_sentence_embedding: ", enable_sentence_embedding, "enable_audio: ", enable_audio)
            seperate_gaussian = sdp.SwissDialectPredictorSeperateGaussians(classifiers={
                                                                    "audio": make_pipeline(GaussianNB()),
                                                                    "sentene_embedding": make_pipeline(
#                                                                        StandardScaler(),
#                                                                        TruncatedSVD(400),
                                                                        GaussianNB()),
                                                                    "byte_pair_tfidf": svm.LinearSVC()},
                                                                normalize_each_vector=False,
                                                                enable_byte_pair_tfidf=enable_byte_pair_tfidf,
                                                                enable_sentance_embedding=enable_sentence_embedding,
                                                                enable_audio=enable_audio,
                                                               last_classifier=svm.LinearSVC(), audio_weight=1)
            seperate_gaussian.fit(train_df)
            prediction_train = seperate_gaussian.predict(train_df)
            print(classification_report(train_df["label"].tolist(), prediction_train))
            prediction_dev = seperate_gaussian.predict(dev_df)
            print(classification_report(dev_df["label"].tolist(), prediction_dev))

enable_byte_pair_tfidf:  True enable_sentence_embedding:  True enable_audio:  True



tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.87      0.89      0.88      3750
          BS       0.88      0.85      0.86      3269
          LU       0.82      0.87      0.84      3390
          ZH       0.90      0.85      0.87      3870

    accuracy                           0.87     14279
   macro avg       0.87      0.87      0.87     14279
weighted avg       0.87      0.87      0.87     14279

              precision    recall  f1-score   support

          BE       0.63      0.59      0.61      1053
          BS       0.80      0.77      0.79      1528
          LU       0.61      0.48      0.54      1017
          ZH       0.65      0.89      0.75       932

    accuracy                           0.69      4530
   macro avg       0.67      0.68      0.67      4530
weighted avg       0.69      0.69      0.68      4530

enable_byte_pair_tfid

In [14]:
etrain_swissbert.iloc[0]["sentence_embedding_first_state"]

array([-5.94933033e-01, -3.14741343e-01, -2.37563998e-01,  2.25697726e-01,
        1.91635162e-01,  7.21708596e-01, -5.51033244e-02, -1.44991800e-01,
        2.61286288e-01,  5.17926335e-01, -2.01473996e-01,  1.44555017e-01,
       -2.11384326e-01, -4.37823862e-01,  8.50655958e-02,  1.30055621e-01,
       -1.59571350e-01, -2.58173913e-01, -4.41073142e-02, -3.10205128e-02,
       -8.72448087e-02, -6.66615605e-01,  1.94224864e-01, -6.16285205e-01,
        3.14802378e-01,  3.69389504e-01, -2.53053904e-01,  2.39723399e-01,
        1.85116202e-01,  1.30352348e-01,  9.32967663e-03, -5.90686560e-01,
        3.99010926e-02, -1.57236010e-01, -3.46788943e-01, -1.90028936e-01,
       -8.91360417e-02, -2.80196704e-02, -8.48822966e-02, -9.22569484e-02,
       -2.65120566e-01, -3.26919705e-02, -1.98382631e-01, -6.85841739e-02,
        1.70206025e-01,  1.56298906e-01, -4.78374183e-01, -2.37082466e-01,
       -2.49887049e-01,  7.28962541e-01,  7.30585307e-02, -2.83880055e-01,
       -7.10680336e-03,  

In [6]:
from binary_swiss_dialect_predictor import BinarySwissDialectPredictor

binary_swiss_predictor = BinarySwissDialectPredictor(  classifiers={"audio": GaussianNB(),
                              "sentene_embedding": GaussianNB(),
                              "byte_pair_tfidf": svm.LinearSVC()},
                 enable_audio=True,
                 enable_sentance_embedding=True,
                 enable_byte_pair_tfidf=True,
                 normalize_each_vector=False,
                 last_classifier=svm.LinearSVC())
binary_swiss_predictor.fit(train_df)
prediction_train = binary_swiss_predictor.predict(train_df)
print(classification_report(train_df["label"].tolist(), prediction_train))
prediction_dev = binary_swiss_predictor.predict(dev_df)
print(classification_report(dev_df["label"].tolist(), prediction_dev))

/Users/raphael/opt/miniconda3/envs/swiss_dial/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(





tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.99      0.98      0.99      3750
          BS       0.99      0.98      0.98      3269
          LU       0.99      0.98      0.98      3390
          ZH       0.96      0.99      0.98      3870

    accuracy                           0.98     14279
   macro avg       0.98      0.98      0.98     14279
weighted avg       0.98      0.98      0.98     14279

              precision    recall  f1-score   support

          BE       0.21      0.19      0.20      1053
          BS       0.93      0.71      0.81      1528
          LU       0.38      0.08      0.13      1017
          ZH       0.41      0.96      0.57       932

    accuracy                           0.50      4530
   macro avg       0.48      0.49      0.43      4530
weighted avg       0.53      0.50      0.47      4530



In [19]:
from binary_swiss_dialect_predictor import BinarySwissDialectPredictor

binary_swiss_predictor = BinarySwissDialectPredictor(  classifiers={"audio": GaussianNB(),
                              "sentene_embedding": GaussianNB(),
                              "byte_pair_tfidf": svm.LinearSVC()},
                 enable_audio=True,
                 enable_sentance_embedding=True,
                 enable_byte_pair_tfidf=True,
                 normalize_each_vector=False,
                 last_classifier=GaussianNB())
binary_swiss_predictor.fit(train_df)
prediction_train = binary_swiss_predictor.predict(train_df)
print(classification_report(train_df["label"].tolist(), prediction_train))
prediction_dev = binary_swiss_predictor.predict(dev_df)
print(classification_report(dev_df["label"].tolist(), prediction_dev))

/Users/raphael/opt/miniconda3/envs/swiss_dial/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(





tf-idf vocab size: 680
              precision    recall  f1-score   support

          BE       0.99      0.98      0.99      3750
          BS       0.99      0.98      0.98      3269
          LU       0.99      0.98      0.98      3390
          ZH       0.96      0.99      0.98      3870

    accuracy                           0.98     14279
   macro avg       0.98      0.98      0.98     14279
weighted avg       0.98      0.98      0.98     14279

              precision    recall  f1-score   support

          BE       0.21      0.19      0.20      1053
          BS       0.93      0.71      0.81      1528
          LU       0.38      0.08      0.13      1017
          ZH       0.41      0.96      0.57       932

    accuracy                           0.50      4530
   macro avg       0.48      0.49      0.43      4530
weighted avg       0.53      0.50      0.47      4530

